In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


In [3]:


# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [4]:


# Preprocess the data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0  # Normalize and reshape
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)  # One-hot encode the labels
y_test = to_categorical(y_test, 10)


In [5]:

# Define the LeNet-5 model
def lenet5():
    model = models.Sequential()

    # Layer 1: Convolutional + Activation + Subsampling (Pooling)
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28, 28, 1), padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

    # Layer 2: Convolutional + Activation + Subsampling (Pooling)
    model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='tanh', padding='valid'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

    # Flattening
    model.add(layers.Flatten())

    # Layer 3: Fully Connected + Activation
    model.add(layers.Dense(120, activation='tanh'))

    # Layer 4: Fully Connected + Activation
    model.add(layers.Dense(84, activation='tanh'))

    # Layer 5: Output layer
    model.add(layers.Dense(10, activation='softmax'))

    return model


In [6]:

# Compile the model
model = lenet5()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\naman\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8175 - loss: 0.6634 - val_accuracy: 0.9505 - val_loss: 0.1773
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9543 - loss: 0.1532 - val_accuracy: 0.9649 - val_loss: 0.1186
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9717 - loss: 0.0959 - val_accuracy: 0.9732 - val_loss: 0.0906
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9811 - loss: 0.0671 - val_accuracy: 0.9741 - val_loss: 0.0850
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9827 - loss: 0.0551 - val_accuracy: 0.9756 - val_loss: 0.0774
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9870 - loss: 0.0433 - val_accuracy: 0.9808 - val_loss: 0.0635
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9890 - loss: 0.0355 - val_accuracy: 0.9812 - val_loss: 0.0644
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9919 - loss: 0.0267 - val_accuracy: 0.

In [8]:

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc * 100:.2f}%")


313/313 - 0s - 1ms/step - accuracy: 0.9843 - loss: 0.0519
Test accuracy: 98.43%


In [9]:
# Save the model as an H5 file
model.save('m2.h5')

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('m2.h5')

def preprocess_frame(frame):
    """Preprocess the frame for the model: grayscale, resize, normalize."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28))
    resized = cv2.bitwise_not(resized)  # Invert for MNIST-like data
    normalized = resized / 255.0
    input_data = normalized.reshape(1, 28, 28, 1)
    return input_data

def predict_digit(frame):
    """Use the model to predict the digit from a webcam frame."""
    input_data = preprocess_frame(frame)
    prediction = model.predict(input_data)
    digit = np.argmax(prediction)  # Get the highest probability digit
    return digit

def display_message(message, frame, pause_time=2000):
    """Display the message on the webcam frame for a short period."""
    cv2.putText(frame, message, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    cv2.imshow('Digit Recognition', frame)
    cv2.waitKey(pause_time)

def run_digit_recognition():
    """Run the webcam loop to predict digits and stop when the correct digit is predicted."""
    cap = cv2.VideoCapture(0)

    print("Starting digit recognition... Press 'q' to quit at any time.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture the frame. Exiting...")
            break

        # Predict the digit
        digit = predict_digit(frame)

        # Display the prediction on the video feed
        cv2.putText(frame, f"Predicted Digit: {digit}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Digit Recognition', frame)

        # Check if 'q' is pressed to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting digit recognition.")
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    """Main function that controls the digit recognition process."""
    while True:
        # Start the digit recognition task
        run_digit_recognition()

        # After task completion, ask if the user wants to restart
        print("\nPrediction completed. Press 'r' to restart or 'q' to quit.")
        key = cv2.waitKey(0) & 0xFF
        if key == ord('r'):
            print("Restarting the digit recognition task...\n")
        elif key == ord('q'):
            print("Exiting program. Goodbye!")
            break

if __name__ == "__main__":
    main()


Starting digit recognition... Press 'q' to quit at any time.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s